In [2]:

import datetime
import os
import io
import requests
import numpy as np


import pandas as pd
from datetime import timedelta
from datetime import datetime as dt
import time

from io import StringIO
import requests
from datetime import datetime as dt, timedelta
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup


In [5]:
class EEX:
    def __init__(self,access_token):
        self.access_token = access_token
        self.current_date = dt.now()
        
    def EUA_futures(self):
        expiration_date = (self.current_date - timedelta(days=0)).strftime("%Y/%m/%d")
        on_date = (self.current_date - timedelta(days=1)).strftime("%Y/%m/%d")
        url = f"https://webservice-eex.gvsi.com/query/json/getChain/gv.pricesymbol/gv.displaydate/gv.expirationdate/tradedatetimegmt/gv.eexdeliverystart/ontradeprice/close/onexchsingletradevolume/onexchtradevolumeeex/offexchtradevolumeeex/openinterest/?optionroot=%22%2FE.FEUA%22&expirationdate={expiration_date}&onDate={on_date}"
    
        headers = {
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Origin": "https://www.eex.com",
        "Referer": "https://www.eex.com/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "cross-site",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
        "sec-ch-ua": "\"Not/A)Brand\";v=\"99\", \"Google Chrome\";v=\"115\", \"Chromium\";v=\"115\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
    }
        print(url)
        response = requests.get(url, headers=headers)
        data = response.json()
        items_list = data['results']['items']
        df = pd.DataFrame(items_list)
        print(df.columns)
        df.drop(columns=["gv.pricesymbol","gv.displaydate","gv.eexdeliverystart"],axis=1,inplace=True)
        df.rename(columns={
                            'tradedatetimegmt':'Date',
                            'gv.expirationdate': 'Future',
                            'ontradeprice': 'Last Price',
                            'close': 'Settlement Price',
                            'onexchsingletradevolume':'Last Volume',
                            'onexchtradevolumeeex':'Volume Trade Registration',
                            'offexchtradevolumeeex':'Volume Exchange',
                            'openinterest':'Open Interest'
                        },inplace=True)
        print(df)

In [6]:
eex_futures = EEX('access_token')
eex_futures.EUA_futures()
    

https://webservice-eex.gvsi.com/query/json/getChain/gv.pricesymbol/gv.displaydate/gv.expirationdate/tradedatetimegmt/gv.eexdeliverystart/ontradeprice/close/onexchsingletradevolume/onexchtradevolumeeex/offexchtradevolumeeex/openinterest/?optionroot=%22%2FE.FEUA%22&expirationdate=2023/12/15&onDate=2023/12/14
Index(['gv.pricesymbol', 'gv.displaydate', 'gv.expirationdate',
       'tradedatetimegmt', 'gv.eexdeliverystart', 'ontradeprice', 'close',
       'onexchsingletradevolume', 'onexchtradevolumeeex',
       'offexchtradevolumeeex', 'openinterest'],
      dtype='object')
        Future                    Date  Last Price  Settlement Price  \
0   12/18/2023  12/14/2023 12:00:00 PM       66.90             66.79   
1    1/29/2024  12/14/2023 12:00:00 PM         NaN             67.14   
2    2/26/2024  12/14/2023 12:00:00 PM         NaN             67.33   
3    3/18/2024  12/14/2023 12:00:00 PM       67.33             67.55   
4    6/24/2024  12/14/2023 12:00:00 PM         NaN             6

In [10]:
class Yahoo:
    def __init__(self, access_token):
        self.access_token = access_token
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
            "Referer": "https://www.hanetf.com/product/30/fund/sparkchange-physical-carbon-eua-etc",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "en-US,en;q=0",
            "Connection": "keep-alive",
        }
        self.interval = '1d'

    def numeric_date(self, date):
        datetime_obj = dt.strptime(date, '%Y-%m-%d')
        timestamp = int(time.mktime(datetime_obj.timetuple()))
        return timestamp

    def yahoo_data(self, symbol):
        
        date1 = (dt.now() - timedelta(days=30)).strftime('%Y-%m-%d')
        date2 = dt.now().strftime('%Y-%m-%d') 
        base_url = 'https://query1.finance.yahoo.com/v7/finance/download/'
        query_url = f'{base_url}{symbol}=X?period1={self.numeric_date(date1)}&period2={self.numeric_date(date2)}&interval={self.interval}&events=history&includeAdjustedClose=true'
        
        response = requests.get(query_url, headers=self.headers)
        data = response.text
        df = pd.read_csv(StringIO(data))
        df = df[['Date','Open']]
        df.rename(columns = {'Open':'EUR_USD'}, inplace = True)
        query_url2 = f'{base_url}GBPUSD=X?period1={self.numeric_date(date1)}&period2={self.numeric_date(date2)}&interval={self.interval}&events=history&includeAdjustedClose=true'
        response2 = requests.get(query_url2, headers=self.headers)
        data2 = response2.text
        df2 = pd.read_csv(StringIO(data2))
        df2 = df2[['Date','Open']]
        df2.rename(columns = {'Open':'GBP_USD'}, inplace = True)
        df3 = pd.merge(df, df2, on='Date', how='inner')
        print(df3)


In [14]:
yahoo = Yahoo('access_token')
yahoo.yahoo_data("EURUSD")
yahoo.yahoo_data("GBPUSD")

          Date   EUR_USD   GBP_USD
0   2023-11-14  1.070183  1.228034
1   2023-11-15  1.087831  1.249407
2   2023-11-16  1.085458  1.242375
3   2023-11-17  1.085376  1.241573
4   2023-11-20  1.090703  1.245407
5   2023-11-21  1.094595  1.250907
6   2023-11-22  1.091822  1.254548
7   2023-11-23  1.089087  1.249110
8   2023-11-24  1.090631  1.253290
9   2023-11-27  1.094044  1.260097
10  2023-11-28  1.095842  1.263568
11  2023-11-29  1.100594  1.271343
12  2023-11-30  1.097370  1.269583
13  2023-12-01  1.089206  1.263185
14  2023-12-04  1.088815  1.271213
15  2023-12-05  1.083776  1.263408
16  2023-12-06  1.079366  1.259493
17  2023-12-07  1.076670  1.255800
18  2023-12-08  1.079214  1.258810
19  2023-12-11  1.076577  1.255745
20  2023-12-12  1.076658  1.255414
21  2023-12-13  1.079867  1.256970
22  2023-12-14  1.088613  1.262164
          Date   EUR_USD   GBP_USD
0   2023-11-14  1.228034  1.228034
1   2023-11-15  1.249407  1.249407
2   2023-11-16  1.242375  1.242375
3   2023-11-17  1.24